Germain Deffontaines - Flavien Lallemand

In [1]:
import bs4 as bs
import urllib.request#Open the cat web data page
cat_data = urllib.request.urlopen('https://simple.wikipedia.org/wiki/Cat').read()#Find all the paragraph html from the web page
cat_data_paragraphs  = bs.BeautifulSoup(cat_data,'lxml').find_all('p')#Creating the corpus of all the web page paragraphs
cat_text = ''#Creating lower text corpus of cat paragraphs
for p in cat_data_paragraphs:
    cat_text += p.text.lower()
    print(cat_text)




cats, also called domestic cats (felis catus), are small, carnivorous mammals, of the family felidae.[3][4]


cats, also called domestic cats (felis catus), are small, carnivorous mammals, of the family felidae.[3][4]
domestic cats are often called 'house cats' when kept as indoor pets.[5] cats have been domesticated (tamed) for nearly 10,000 years.[6]


cats, also called domestic cats (felis catus), are small, carnivorous mammals, of the family felidae.[3][4]
domestic cats are often called 'house cats' when kept as indoor pets.[5] cats have been domesticated (tamed) for nearly 10,000 years.[6]
they are one of the most popular pets in the world. they are kept by humans for hunting rodents and as companions.


cats, also called domestic cats (felis catus), are small, carnivorous mammals, of the family felidae.[3][4]
domestic cats are often called 'house cats' when kept as indoor pets.[5] cats have been domesticated (tamed) for nearly 10,000 years.[6]
they are one of the most popular 

In [2]:
import re
cat_text = re.sub(r'\s+', ' ',re.sub(r'\[[0-9]*\]', ' ', cat_text))

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/germain/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import nltk
cat_sentences = nltk.sent_tokenize(cat_text)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
def chatbot_answer(user_query):
    
    #Append the query to the sentences list
    cat_sentences.append(user_query)    #Create the sentences vector based on the list
    vectorizer = TfidfVectorizer()
    sentences_vectors = vectorizer.fit_transform(cat_sentences)
    
    #Measure the cosine similarity and take the second closest index because the first index is the user query
    vector_values = cosine_similarity(sentences_vectors[-1], sentences_vectors)
    answer = cat_sentences[vector_values.argsort()[0][-2]]    #Final check to make sure there are result present. If all the result are 0, means the text input by us are not captured in the corpus
    input_check = vector_values.flatten()
    input_check.sort()
    
    if input_check[-2] == 0:
        return "Please Try again"
    else: 
        return answer

In [7]:
print("Hello, I am the Cat Chatbot. What is your meow questions?:")
while(True):
    query = input().lower()
    if query not in ['bye', 'good bye', 'take care']:
        print("Cat Chatbot: ", end="")
        print(chatbot_answer(query))
        cat_sentences.remove(query)
    else:
        print("See You Again")
        break

Hello, I am the Cat Chatbot. What is your meow questions?:
Cat Chatbot: polydactyly means many digits, or many "fingers" from poly (many) and dactyl (digit).
Cat Chatbot: here is how to do it: what you have done is exactly what the queen would do in the wild.
Cat Chatbot: here is how to do it: what you have done is exactly what the queen would do in the wild.
Cat Chatbot: Please Try again
Cat Chatbot: most cats have only four to five toes per paw, depending on whether it is the front or back paw.
Cat Chatbot: Please Try again


KeyboardInterrupt: Interrupted by user